<a href="https://colab.research.google.com/github/L-200/cd_speedrun-eldenring/blob/main/Limpeza_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Bibliotecas e configuração necessárias

In [1]:
import pandas as pd
import numpy as np

# Configuração para visualizar todas as colunas no notebook
pd.set_option('display.max_columns', None)

# 2) Funções de Limpeza

Funções que realizam a limpeza para o melhor desempenho dos Modelos de Aprendizado


In [2]:
def carregar_e_padronizar_dados(caminho_arquivo):
    """
    Carrega o CSV, converte colunas de data e ajusta tipos numéricos
    """
    df = pd.read_csv(caminho_arquivo)

    # Conversão de datas (lidando com formatos mistos e timezones)
    cols_data = ['run_date', 'video_data_publicacao']
    for col in cols_data:
        # utc=True ajuda a unificar datas com e sem fuso horário (YouTube vs Bilibili)
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

    # Extrair apenas a data (sem hora) para facilitar análises temporais
    df['data_publicacao_clean'] = df['video_data_publicacao'].dt.date

    return df


def limpar_colunas_irrelevantes(df):
    """
    Remove colunas que não servem como features para os modelos
    (IDs, Links ou colunas com muitos dados faltantes/sentinelas).
    """
    # Lista de colunas para remover (identificadores únicos ou textos não processados)
    cols_to_drop = [
        'video_link',
        'run_player',      # Alta cardinalidade, nome do jogador
        'plataforma',      # Se quiser manter, precisará fazer One-Hot Encoding depois
        'ChannelID',
        'video_data_publicacao' # Já criamos a versão clean/numérica se necessário
    ]

    df_clean = df.drop(columns=cols_to_drop, errors='ignore')

    # Tratamento específico para os dados do Bilibili que vieram como -1 (sentinela de erro/nulo)
    # Substituindo -1 por NaN nas colunas numéricas para não enviesar a média/regressão
    cols_numericas = df_clean.select_dtypes(include=[np.number]).columns
    for col in cols_numericas:
        # Evita substituir na coluna 'hype' ou outras binárias se existirem
        if col != 'hype':
            df_clean[col] = df_clean[col].replace(-1.0, np.nan)

    # Preenchimento de nulos com a mediana (estratégia comum para não perder linhas)
    df_clean = df_clean.fillna(df_clean.median(numeric_only=True))

    return df_clean

# 3) Criação da Categoria Alvo

Foi decidido gerar a categoria "Hype", que é composta pelos 33% de vídeos com mais views para ser a nossa Categoria Alvo.

In [3]:
def criar_categoria_hype(df):
    """
    Cria a variável alvo 'hype' para a etapa de Classificação
    Regra: 1 (Hype) se estiver entre os 33% vídeos mais vistos, 0 (Normal) caso contrário
    """
    # Calculando o corte (percentil 67 = top 33%)
    corte_hype = df['video_views'].quantile(0.67)

    # Criando a coluna binária (Target)
    df['hype'] = np.where(df['video_views'] >= corte_hype, 1, 0)

    print(f"Corte de visualizações para ser 'Hype': {corte_hype:,.0f}")
    print(f"Distribuição das classes:\n{df['hype'].value_counts(normalize=True)}")

    return df

# 4) Execução das funções

In [4]:
# Carregamento
df_raw = carregar_e_padronizar_dados('dados_etapa_anterior.csv')

# Criação do Target (Obrigatório para etapa de Classificação)
df_hype = criar_categoria_hype(df_raw)

# Limpeza Final (Preparação para dividir em Treino/Teste)
df_final = limpar_colunas_irrelevantes(df_hype)

# Visualizando o resultado
df_final.head()

Corte de visualizações para ser 'Hype': 24,014
Distribuição das classes:
hype
0    0.666667
1    0.333333
Name: proportion, dtype: float64


,run_date,run_time_seconds,VideoAgeDays,video_views,video_likes,video_comentarios,ViewsPerDay,EngagementRate,CurrentSubscribers,Views_5d_Antes,Likes_5d_Antes,NumVideos_5d_Antes,Views_5d_Depois,Likes_5d_Depois,NumVideos_5d_Depois,danmaku,coins,shares,favorites,data_publicacao_clean,hype
0,2022-06-25 00:00:00+00:00,3979.0,1202.0,1114.0,25.0,3.0,0.926789,2.513465,928.0,0.0,0.0,0.0,0.0,0.0,0.0,2380.5,3464.5,1116.5,2303.0,2022-06-25,0
1,2022-07-02 00:00:00+00:00,3908.0,1194.0,5098.0,141.0,27.0,4.269682,3.295410,12100.0,1556.0,45.0,1.0,7534.0,283.0,4.0,2380.5,3464.5,1116.5,2303.0,2022-07-02,0
2,2022-07-04 00:00:00+00:00,3880.0,1192.0,2122.0,92.0,19.0,1.780201,5.230914,12100.0,6654.0,186.0,2.0,5412.0,191.0,3.0,2380.5,3464.5,1116.5,2303.0,2022-07-04,0
3,2022-07-05 00:00:00+00:00,3855.0,1191.0,1215.0,47.0,12.0,1.020151,4.855967,12100.0,7220.0,233.0,2.0,4197.0,144.0,2.0,2380.5,3464.5,1116.5,2303.0,2022-07-06,0
4,2022-07-06 00:00:00+00:00,3804.0,1190.0,2276.0,75.0,19.0,1.912605,4.130053,12100.0,10356.0,349.0,4.0,1629.0,40.0,1.0,2380.5,3464.5,1116.5,2303.0,2022-07-07,0


# 5) Salvando o df gerado

In [5]:
nome_arquivo_saida = 'dados_para_modelagem.csv'
df_final.to_csv(nome_arquivo_saida, index=False)

print(f"Arquivo '{nome_arquivo_saida}' salvo com sucesso!")

Arquivo 'dados_para_modelagem.csv' salvo com sucesso!
